<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/IOU%E6%A4%9C%E8%A8%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
!pip install kaggle
!apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
!mkdir input output

In [5]:
!kaggle competitions download -c foursquare-location-matching
!unzip foursquare-location-matching.zip -d input/foursquare-location-matching

 96% 152M/159M [00:00<00:00, 269MB/s]
100% 159M/159M [00:00<00:00, 247MB/s]
Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [6]:
!pip install Levenshtein
!pip install pykakasi
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 4.9 MB/s 
     |████████████████████████████████| 1.8 MB 69.2 MB/s 
     |████████████████████████████████| 103 kB 53.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 5.1 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15564 sha256=0b6f5e86f7a05e4b0917bd205e0872844f8f6ecc17ea2cb97b105859afccddd1
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# change directory

In [7]:
%cd /content/output

/content/output


# start code

In [8]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import Levenshtein
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pykakasi
from tqdm.notebook import tqdm
import math

from fuzzywuzzy import fuzz


tqdm.pandas()

## Conversion kanji to alphabet

In [9]:
def convert_japanese_alphabet(df: pd.DataFrame):
    kakasi = pykakasi.kakasi()
    kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
    kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
    kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
    conversion = kakasi.getConverter()

    def convert(row):
        for column in ["name", "address", "city", "state"]:
            try:
                row[column] = conversion.do(row[column])
            except:
                pass
        return row

    df[df["country"] == "JP"] = df[df["country"] == "JP"].progress_apply(convert, axis=1)
    return df

## start 

In [10]:
## Parameters
is_debug = False
SEED = 2022
num_neighbors = 10
num_neighbors2 = 5
num_neighbors3 = 22
num_split = 10
# feat_columns = ['name', 'address', 'city', 
#             'state', 'zip', 'url', 
#            'phone', 'categories', 'country']
feat_columns = ['dist', 'name', 'address', 'city', 
            'state', 'zip', 'url', 
           'phone', 'categories', 'country']
vec_columns = ['name', 'categories', 'address', 
               'state', 'url', 'country']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

In [11]:
%load_ext Cython

In [12]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [13]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 

# get manhattan distance
def manhattan(lat1, long1, lat2, long2):
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)

# get haversine distance
def vectorized_haversine(lats1, lats2, longs1, longs2):
    radius = 6371
    dlat=np.radians(lats2 - lats1)
    dlon=np.radians(longs2 - longs1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lats1)) \
        * np.cos(np.radians(lats2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c
    return d

In [14]:
def recall_simple(df):
    threshold = 2
    
    val2id_d = {}
    for col in rec_columns:
        temp_df = df[['id', col]]
        temp_df[col] = temp_df[col].str.lower()
        val2id = temp_df.groupby(col)['id'].apply(set).to_dict()
        val2id_d[col] = val2id
        del val2id
    
    cus_ids = []
    match_ids = []
    for vals in tqdm(df[rec_columns + ['id']].fillna('null').values):
        cus_id = vals[-1]
        match_id = []
        
        rec_match_count = []
        for i in range(len(rec_columns)):
            col = rec_columns[i]
            
            if vals[i] != 'null':
                rec_match_count += list(val2id_d[col][vals[i].lower()])
        rec_match_count = dict(Counter(rec_match_count))
        
        for k, v in rec_match_count.items():
            if v > threshold:
                match_id.append(k)
        
        cus_ids += [cus_id] * len(match_id)
        match_ids += match_id
    
    train_df = pd.DataFrame()
    train_df['id'] = cus_ids
    train_df['match_id'] = match_ids
    train_df = train_df.drop_duplicates()
    del cus_ids, match_ids
    
    num_data = len(train_df)
    num_data_per_id = num_data / train_df['id'].nunique()
    print('Num of data: %s' % num_data)
    print('Num of data per id: %s' % num_data_per_id)
    
    return train_df

In [15]:
# data.groupby('categories')['categories'].count().sort_values(ascending=False)[:500]

In [16]:
def recall_knn(df, Neighbors = 10, Neighbors2 = 10, Neighbors3 = 10 ):
    print('Start knn grouped by country')
    train_df_country = []
    for country, country_df in tqdm(df.groupby('country')):
        country_df = country_df.reset_index(drop = True)

        neighbors = min(len(country_df), Neighbors)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(country_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                country_df.index)
        dists, nears = knn.kneighbors(
            country_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(country_df[['latitude','longitude']],
        #         country_df.index)
        # dists, nears = knn.kneighbors(
        #     country_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = country_df[['id']]
            cur_df['match_id'] = country_df['id'].values[nears[:, k]]
            cur_df['kdist_country'] = dists[:, k]
            cur_df['kneighbors_country'] = k
            
            train_df_country.append(cur_df)
    train_df_country = pd.concat(train_df_country)
    
    #-------------------------------------------------
    print('Start knn grouped by categories')
    sel = set(data.groupby('categories')['categories'].count().sort_values(ascending=False)[:500].keys())
    train_df_categories = []
  
    for categories, categories_df in tqdm(df.groupby('categories')):
        if categories not in sel : continue
        categories_df = categories_df.reset_index(drop = True)

        neighbors = min(len(categories_df), Neighbors2)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(categories_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                categories_df.index)
        dists, nears = knn.kneighbors(
            categories_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        # knn.fit(categories_df[['latitude','longitude']],
        #         categories_df.index)
        # dists, nears = knn.kneighbors(
        #     categories_df[['latitude', 'longitude']], 
        #     return_distance = True)

        for k in range(neighbors):            
            cur_df = categories_df[['id']]
            cur_df['match_id'] = categories_df['id'].values[nears[:, k]]
            cur_df['kdist_categories'] = dists[:, k]
            cur_df['kneighbors_categories'] = k
            
            train_df_categories.append(cur_df)
    train_df_categories = pd.concat(train_df_categories)
    #-------------------------------------------------
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors3)
    knn.fit(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180), df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180))
    # knn.fit(df[['latitude','longitude']], df.index)
    # dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors3):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.merge(train_df_country,
                                 on = ['id', 'match_id'],
                                 how = 'outer')
    #-------------------------------------------------
    train_df = train_df.merge(train_df_categories,
                                on = ['id', 'match_id'],
                                how = 'outer')

    del train_df_country
    
    return train_df

In [17]:
def add_features(df):    
    for col in tqdm(feat_columns):
        if col == 'dist':
            lat = data.loc[df['id']]['latitude'].values
            match_lat = data.loc[df['match_id']]['latitude'].values
            lon = data.loc[df['id']]['longitude'].values
            match_lon = data.loc[df['match_id']]['longitude'].values
            df['latdiff'] = (lat - match_lat)
            df['londiff'] = (lon - match_lon)
            df['manhattan'] = manhattan(lat, lon, match_lat, match_lon)
            df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
            df['haversine'] = vectorized_haversine(lat, match_lat, lon, match_lon)
            continue
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        
        if col in vec_columns:
            tv_fit = tfidf_d[col]
            indexs = [id2index_d[i] for i in df['id']]
            match_indexs = [id2index_d[i] for i in df['match_id']]                    
            df[f'{col}_sim'] = tv_fit[indexs].multiply(tv_fit[match_indexs]).\
                                            sum(axis = 1).A.ravel()
        
        geshs = []
        levens = []
        jaros = []
        lcss = []
        fuzwuz = []
        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan' and match_s != 'nan':                    
                geshs.append(difflib.SequenceMatcher(None, s, match_s).ratio())
                levens.append(Levenshtein.distance(s, match_s))
                jaros.append(Levenshtein.jaro_winkler(s, match_s))
                lcss.append(LCS(str(s), str(match_s)))
                fuzwuz.append(fuzz.ratio(s, match_s))
            else:
                geshs.append(np.nan)
                levens.append(np.nan)
                jaros.append(np.nan)
                lcss.append(np.nan)
                fuzwuz.append(np.nan)
        
        df[f'{col}_gesh'] = geshs
        df[f'{col}_leven'] = levens
        df[f'{col}_jaro'] = jaros
        df[f'{col}_lcs'] = lcss
        df[f'{col}_fuz'] = fuzwuz
        
        if col not in ['phone', 'zip']:
            df[f'{col}_len'] = list(map(len, col_values))
            df[f'match_{col}_len'] = list(map(len, matcol_values)) 
            df[f'{col}_len_diff'] = np.abs(df[f'{col}_len'] - df[f'match_{col}_len']) /\
                                        df[f'{col}_len'] 
            df[f'{col}_nleven'] = df[f'{col}_leven'] / \
                                    df[[f'{col}_len', f'match_{col}_len']].max(axis = 1)
            
            df[f'{col}_nlcsk'] = df[f'{col}_lcs'] / df[f'match_{col}_len']
            df[f'{col}_nlcs'] = df[f'{col}_lcs'] / df[f'{col}_len']
            
            df = df.drop(f'{col}_len', axis = 1)
            df = df.drop(f'match_{col}_len', axis = 1)
            gc.collect()
            
    return df

In [18]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def analysis(df):
    print('Num of data: %s' % len(df))
    print('Num of unique id: %s' % df['id'].nunique())
    print('Num of unique poi: %s' % df['point_of_interest'].nunique())
    
    poi_grouped = df.groupby('point_of_interest')['id'].count().reset_index()
    print('Mean num of unique poi: %s' % poi_grouped['id'].mean())

In [19]:
## Data load
data_root = '../input/foursquare-location-matching'
data = pd.read_csv(os.path.join(data_root, 'train.csv'))

if is_debug:
    data = data.sample(n = 10000, random_state = SEED)
    data = data.reset_index(drop = True)

In [20]:
data[data["country"] == "JP"].head(2)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
12,E_0000c566a81ea1,つじ田 味噌の章,35.694796,139.767348,神田小川町1-1,千代田区,東京都,101-0052,JP,https://tsukemen-tsujita.com,366597676,Ramen Restaurants,P_5fcd8cd98fea2b
17,E_00014f107dc217,ローソン 弘前撫牛子店,40.623279,140.494751,撫牛子2-10ｰ2,弘前市,青森県,036-8075,JP,http://www.lawson.co.jp,172325353,Convenience Stores,P_d0bb3bdad1ab4f


In [21]:
# data = convert_japanese_alphabet(data)

In [22]:
data[data["country"] == "JP"].head(2)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
12,E_0000c566a81ea1,つじ田 味噌の章,35.694796,139.767348,神田小川町1-1,千代田区,東京都,101-0052,JP,https://tsukemen-tsujita.com,366597676,Ramen Restaurants,P_5fcd8cd98fea2b
17,E_00014f107dc217,ローソン 弘前撫牛子店,40.623279,140.494751,撫牛子2-10ｰ2,弘前市,青森県,036-8075,JP,http://www.lawson.co.jp,172325353,Convenience Stores,P_d0bb3bdad1ab4f


In [23]:
## Data split
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(data, 
                                                data['point_of_interest'], 
                                                data['point_of_interest'])):
    data.loc[val_idx, 'set'] = i  # 1-i 逆にした

print('Num of train data: %s' % len(data))
print(data['set'].value_counts())

valid_data = data[data['set'] == 0]
train_data = data[data['set'] == 1]

print('Train data: ')
analysis(train_data)
print('Valid data: ')
analysis(valid_data)

train_poi = train_data['point_of_interest'].unique().tolist()
valid_poi = valid_data['point_of_interest'].unique().tolist()

print(set(train_poi) & set(valid_poi))

train_ids = train_data['id'].unique().tolist()
valid_ids = valid_data['id'].unique().tolist()
      
print(set(train_ids) & set(valid_ids))
      
tv_ids_d = {}
tv_ids_d['train_ids'] = train_ids
tv_ids_d['valid_ids'] = valid_ids

np.save('tv_ids_d.npy', tv_ids_d)

del train_data, valid_data
gc.collect()

data = data.set_index('id')
data = data.loc[tv_ids_d['train_ids']]
data = data.reset_index()

Num of train data: 1138812
1.0    569406
0.0    569406
Name: set, dtype: int64
Train data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369987
Mean num of unique poi: 1.5389892077289202
Valid data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369985
Mean num of unique poi: 1.5389975269267673
set()
set()


In [24]:
len(data)

569406

In [25]:
%%time
## Train data generated by knn
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in vec_columns:
    tfidf = TfidfVectorizer()
    tv_fit = tfidf.fit_transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

train_data = recall_knn(data, num_neighbors, num_neighbors2, num_neighbors3)

data = data.set_index('id')
ids = train_data['id'].tolist()
match_ids = train_data['match_id'].tolist()

poi = data.loc[ids]['point_of_interest'].values
match_poi = data.loc[match_ids]['point_of_interest'].values

train_data['label'] = np.array(poi == match_poi, dtype = np.int8)
del poi, match_poi, ids, match_ids
gc.collect()

print('Num of unique id: %s' % train_data['id'].nunique())
print('Num of train data: %s' % len(train_data))
print('Pos rate: %s' % train_data['label'].mean())
print("rate = ", len(train_data)/13928057)
print(train_data.sample(5))

Start knn grouped by country


  0%|          | 0/210 [00:00<?, ?it/s]

Start knn grouped by categories


  0%|          | 0/32390 [00:00<?, ?it/s]

Start knn
Num of unique id: 569406
Num of train data: 13910594
Pos rate: 0.07449042075413889
rate =  0.9987461998468271
                       id          match_id     kdist  kneighbors  \
4639643  E_25d223267cd7bb  E_bcf49bde1435e4  0.000152         8.0   
7497744  E_2ad1fc03ade29c  E_fb594c86c03cb0  0.000103        13.0   
1203113  E_1ccbcc14e9f3b3  E_16df0fb274f21b  0.000008         2.0   
8300353  E_9390ddcf422523  E_2ddb9c48d509d4  0.004275        14.0   
3907809  E_dcd9b6e0261a23  E_38be2a42ca87a4  0.000111         6.0   

         kdist_country  kneighbors_country  kdist_categories  \
4639643            NaN                 NaN               NaN   
7497744            NaN                 NaN               NaN   
1203113       0.000008                 2.0               NaN   
8300353            NaN                 NaN               NaN   
3907809       0.000080                 6.0               NaN   

         kneighbors_categories  label  
4639643                    NaN      0  


In [26]:
sel = set(data.groupby('categories')['categories'].count().sort_values(ascending=False)[:300].keys())
sel

{'ATMs',
 'Accessories Stores',
 'Advertising Agencies',
 'Airport Lounges',
 'Airport Services',
 'Airport Terminals',
 'Airports',
 'American Restaurants',
 'Arcades',
 'Art Galleries',
 'Arts & Crafts Stores',
 'Asian Restaurants',
 'Assisted Living',
 'Athletics & Sports',
 'Auditoriums',
 'Auto Dealerships',
 'Auto Dealerships, Automotive Shops',
 'Auto Garages',
 'Auto Workshops',
 'Auto Workshops, Automotive Shops, Auto Garages',
 'Automotive Shops',
 'BBQ Joints',
 'Bagel Shops',
 'Bakeries',
 'Banks',
 'Banks, ATMs',
 'Bars',
 'Baseball Fields',
 'Baseball Stadiums',
 'Beaches',
 'Bed & Breakfasts',
 'Beer Gardens',
 'Big Box Stores',
 'Big Box Stores, Supermarkets, Grocery Stores',
 'Bike Rentals / Bike Shares',
 'Bike Shops',
 'Bistros',
 'Boats or Ferries',
 'Bookstores',
 'Border Crossings',
 'Boutiques',
 'Bowling Alleys',
 'Brazilian Restaurants',
 'Breakfast Spots',
 'Breweries',
 'Bridal Shops',
 'Bridges',
 'Bubble Tea Shops',
 'Buddhist Temples',
 'Buffets',
 'Buildi

In [27]:
sum(train_data['kdist'].isna()), sum(train_data['kdist'].notna()),  len(train_data)

(1383662, 12526932, 13910594)

In [28]:
# 23341707
# 11952220

In [29]:
train_data.head()

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,kdist_categories,kneighbors_categories,label
0,E_000001272c6c5d,E_000001272c6c5d,0.0,0.0,0.0,0.0,0.0,0.0,1
1,E_00009ab517afac,E_00009ab517afac,0.0,0.0,0.0,0.0,0.0,0.0,1
2,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,0.0,0.0,0.0,1
3,E_0000c566a81ea1,E_0000c566a81ea1,0.0,0.0,0.0,0.0,0.0,0.0,1
4,E_0000d9e584ed9f,E_0000d9e584ed9f,0.0,0.0,0.0,0.0,0.0,0.0,1


## 6/18 追加（反映チェック必要） kdistに距離を集約

In [30]:
pos = train_data['kdist_country'].notna()
train_data.loc[pos, 'kdist'] = train_data[pos]['kdist_country']

In [31]:
pos = train_data['kdist_categories'].notna()
train_data.loc[pos, 'kdist'] = train_data[pos]['kdist_categories']

In [32]:
sum(train_data['kdist'].isna())

0

In [33]:
sel = ['id', 'match_id', 'kdist', 'kneighbors',
       'kneighbors_country', 'kneighbors_categories',
       'label']
train_data = train_data[sel]

In [34]:
## Eval
data = data.reset_index()

id2poi = get_id2poi(data)
poi2ids = get_poi2ids(data)

eval_df = pd.DataFrame()
eval_df['id'] = data['id'].unique().tolist()
eval_df['match_id'] = eval_df['id']
print('Unique id: %s' % len(eval_df))

eval_df_ = train_data[train_data['label'] == 1][['id', 'match_id']]
eval_df = pd.concat([eval_df, eval_df_])

eval_df = eval_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
eval_df['matches'] = eval_df['match_id'].apply(lambda x: ' '.join(set(x)))
print('Unique id: %s' % len(eval_df))

iou_score = get_score(eval_df)
print('IoU score: %s' % iou_score)

Unique id: 569406
Unique id: 569406
IoU score: 0.9364244974115902


In [35]:
print('IoU score: %s' % iou_score, num_neighbors, num_neighbors2, num_neighbors3,  len(train_data)/13928057)

IoU score: 0.9364244974115902 10 5 22


In [36]:
 # IoU score: 0.9210875384608563 20 20 1.0 --- original

In [37]:
#IoU score: 0.936960291817873 1,39
#IoU score: 0.9344528470303082 5 35
#IoU score: 0.9375555341479278 3 40
#IoU score: 0.9375531925264253 2 40
#IoU score: 0.9375520217156739 1 40 1.6352776270229221
#IoU score: 0.9375555341479278 3 40 1.6353082127679404
#----------->  not radian #IoU score: 0.9375596682994936 3 40 1.6405100151442515
#IoU score: 0.9429661185642001 3 40
#IoU score: 0.9404926843925454 3 35 1.4721750492548962
#IoU score: 0.9352833892684943 10 20
#IoU score: 0.9434660236746832 5 30 1.5175960293671975
#IoU score: 0.9345203666132559 10 5 20 0.918320193548892
#IoU score: 0.9389112753713773 10 5 25 1.1197341452580212
#IoU score: 0.9347000717321834 10 5 20 0.920687932279427
#IoU score: 0.9364244974115902 10 5 22 0.9987461998468271

In [38]:
## Add features
count = 0
start_row = 0

data = data.set_index('id')
unique_id = train_data['id'].unique().tolist()
num_split_id = len(unique_id) // num_split
for k in range(1, num_split + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < num_split:
        cur_id = unique_id[start_row : end_row]
        cur_data = train_data[train_data['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = train_data[train_data['id'].isin(cur_id)]
    
    cur_data = add_features(cur_data)
    print(cur_data.shape)
    print(cur_data.sample(1))
    
    cur_data.to_csv('train_data%s.csv' % k, index = False)    
    print(cur_data.columns)
    start_row = end_row
    count += len(cur_data)
    
    del cur_data
    gc.collect()
    
print(count)

Current split: 1


  0%|          | 0/10 [00:00<?, ?it/s]

(1390963, 91)
                       id          match_id     kdist  kneighbors  \
5175315  E_169debcf56ef09  E_4d6107ae394359  0.000065         9.0   

         kneighbors_country  kneighbors_categories  label   latdiff   londiff  \
5175315                 7.0                    NaN      0 -0.001073 -0.004522   

         manhattan  ...  country_sim  country_gesh  country_leven  \
5175315   0.005596  ...          1.0           1.0            0.0   

         country_jaro  country_lcs  country_fuz  country_len_diff  \
5175315           1.0          2.0        100.0               0.0   

         country_nleven  country_nlcsk  country_nlcs  
5175315             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_d

  0%|          | 0/10 [00:00<?, ?it/s]

(1391400, 91)
                       id          match_id     kdist  kneighbors  \
8612488  E_1ffb5535300bb7  E_58859533e0824d  0.000039        15.0   

         kneighbors_country  kneighbors_categories  label   latdiff   londiff  \
8612488                 NaN                    NaN      0 -0.001177  0.001917   

         manhattan  ...  country_sim  country_gesh  country_leven  \
8612488   0.003094  ...          1.0           1.0            0.0   

         country_jaro  country_lcs  country_fuz  country_len_diff  \
8612488           1.0          2.0        100.0               0.0   

         country_nleven  country_nlcsk  country_nlcs  
8612488             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_d

...

(1390302, 91)
                        id          match_id     kdist  kneighbors  \
13345295  E_cc7077a9f87d3d  E_27f1f6b14a497f  0.117448         NaN   

          kneighbors_country  kneighbors_categories  label   latdiff  \
13345295                 NaN                    2.0      0  2.415227   

           londiff  manhattan  ...  country_sim  country_gesh  country_leven  \
13345295  6.854109   9.269336  ...          0.0           0.0            2.0   

          country_jaro  country_lcs  country_fuz  country_len_diff  \
13345295           0.0          0.0          0.0               0.0   

          country_nleven  country_nlcsk  country_nlcs  
13345295             1.0            0.0           0.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', '

  0%|          | 0/10 [00:00<?, ?it/s]

(1391098, 91)
                        id          match_id     kdist  kneighbors  \
13266520  E_e5f99ce3cb2e5c  E_e1a237f359861c  0.000704         NaN   

          kneighbors_country  kneighbors_categories  label  latdiff   londiff  \
13266520                 NaN                    4.0      0 -0.00456 -0.080576   

          manhattan  ...  country_sim  country_gesh  country_leven  \
13266520   0.085136  ...          1.0           1.0            0.0   

          country_jaro  country_lcs  country_fuz  country_len_diff  \
13266520           1.0          2.0        100.0               0.0   

          country_nleven  country_nlcsk  country_nlcs  
13266520             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'na

  0%|          | 0/10 [00:00<?, ?it/s]

(1390889, 91)
                       id          match_id     kdist  kneighbors  \
1103436  E_f01ad7adb92c45  E_e6f06761c2e451  0.000036         1.0   

         kneighbors_country  kneighbors_categories  label   latdiff   londiff  \
1103436                 1.0                    NaN      1  0.002068 -0.000442   

         manhattan  ...  country_sim  country_gesh  country_leven  \
1103436    0.00251  ...          1.0           1.0            0.0   

         country_jaro  country_lcs  country_fuz  country_len_diff  \
1103436           1.0          2.0        100.0               0.0   

         country_nleven  country_nlcsk  country_nlcs  
1103436             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kneighbors_country',
       'kneighbors_categories', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_d

In [ ]:
!cp * /content/drive/MyDrive/datas/foursquare/traindata4